## 1.1 编程的基本元素

我们在看待一门编程语言时，不应该仅仅把它的当作是**让计算机完成某些任务的指令**，而是应该考虑如何用这门语言来组织、表达我们关于**计算过程**的想法，应该关注这门语言是如何将简单的想法**组合成**复杂的想法的。编程语言一般都是通过下面三种机制来完成这一目标：

1. 基本表达式（primitive expressions)
2. 组合方法（means of combination）
3. 抽象方法（means of abstraction）

**基本表达式**用于表示语言所关注的最简单、最基础的实体；通过**组合方法**将简单的元素组合称复合元素（compound elements）；复合元素可以通过**抽象方法**被抽象成带有名字的实体，从而可以像简单的基本元素一样直接被操作。

上面提到的基本元素、简单元素和复合元素，总的来说在编程中它们可以被分成两类：**过程（procedure）**和**数据（data）**，数据是我们希望操作的内容，而过程则是对数据处理规则的描述。（前面已经说过，我们将会看到它们两者之间并没有本质差别）

#### 1.1.1 表达式

当在Scheme解释器中输入数字**表达式(expression)**时，它会返回对该表达式**求值（evaluating）**的结果。数字表达式通常会与代表基本过程的表达式一起出现，完成对数字的数学运算：

In [1]:
(+ 137 349)

486

In [2]:
(* 5 99)

495

In [3]:
(/ 10 5)

2

如上所示，用`()`区分出来的一组表达式的列表，称为**组合式（combinations）**，其中最左边的元素称为**运算符（operator）**，其余元素称为**运算对象（operands）**。组合式的值是通过将所有运算对象作为参数（**数据**）应用到运算符所指定的**过程**中所求得的。

这种将运算符放在最前面的表示方法称为**前缀表示法（prefix notation）**，相对于常见的中缀表示法（运算符放在中间），它有以下优点：

**1. 一个运算符可以接受任意多个运算对象作为参数：**

In [4]:
(+ 21 35 12 7)

75

**2. 能够更直观地表示含有嵌套的组合式，因为组合式本身变成了运算对象（对应上面提到的第二种机制：组合方法）：**

In [5]:
(+ (* 3 5) (- 10 6))

19

In [6]:
(+ (* 3
      (+ (* 2 4)
         (+ 3 5)))
   (+ (- 10 7)
      6))

57

#### 1.1.2 命名和环境

编程语言提供可以对计算对象进行命名的方法，得到**值**为该计算对象的**变量**。因此变量就是被命名过的值（这里的值可以是任何（基本/组合）表达式的求值）。Scheme中用`define`对元素进行命名：

In [7]:
(define size 2)
size

2

In [8]:
(* 5 size)

10

`define`是Scheme中最简单的抽象方法（第三种机制），因为它可以让我们用简单的名字来指代由许多元素组成的复合元素。复合元素（或组合式）所构成的计算对象可能有具有非常复杂的内部结构，抽象方法使重复利用变得简洁、方便：

In [9]:
(define pi 3.1415926)
(define radius 10)
(define circumference (* 2 pi radius))
circumference

62.831852

抽象方法将值绑定到名称上供后续使用，意味着解释器需要将这些**名称-值**的关联保存在记忆中，这些记忆称为**环境（environment）**（更确切地说，这里是**全局环境（global environment）**）。

#### 1.1.3 组合式求值

解释器按照下面的规则对组合式进行求值：

1. 对组合式的子表达式求值；
2. 将作为运算对象的所有子表达式的值应用到最左边的运算符子表达式。

先看第一步，首先需要对组合式的各个元素进行求值，因为各元素本身也有可能是组合式，因此第一步从本质上来说就是递归过程，在执行这一步时可能需要重新调用该规则本身。

以下面的计算为例：

In [10]:
(* (+ 2 (* 4 6)) (+ 3 5 7))

390

上面的计算过程需要执行4次对组合式求值的流程，可以通过下面的树状图来表示：

![Figure 1.1](imgs/figure-1.1.png)

树状图的末端代表运算符或数字（基本表达式），有分支的节点代表对子表达式求值的结果。可以看出在自下而上进行计算的过程中，规则1被递归地调用了3次。

然后我们来看根据规则1我们来到树状图的最末端，不再是对组合式求值，而是对基本表达式如数字、内置运算符或变量进行求值，此时需要依照下面的三条准则来执行：

1. 数字表达式的值就是数字所代表的值；
2. 内置运算符就是完成对应运算操作的机器指令；
3. 变量的值是在解释器记忆环境中与其名字绑定的值。

其中第2条准则实际上是第3条的特殊形式，因为`+ *`等运算符实际上也是绑定在解释器记忆环境中的名称（符号），只不过它们所绑定的对象是进行相应数学运算的机器指令序列。

---

需要注意的是，上面的求值规则中并不包括`define`表达式，因为对`(define x 3)`进行求值并不是将`x 3`作为参数传递给`define`，`define`是将名称`x`绑定到一个值`3`上，也就是说，`(define x 3)`与前面例子中的`(* 4 3)`不同，**它并不是一个复合表达式**。像这种一般求值规则之外的例外情况，称为**特殊形式（special forms）**，`define`是到目前为止我们遇到的唯一一个特殊形式。所有的特殊形式都有它们自己的求值规则。而各种各样的表达式包括各自的求值规则（这里包含一般的表达式和特殊形式）组成了编程语言的语法。

Lisp语言的优点在于它拥有非常简单的语法，Lisp表达式的求值可以总结为：**由简单的、一般形式的表达式的求值规则和少量特殊形式的求值规则组成**。

#### 1.1.4 组合过程

到现在为止，我们已经学习了Lisp的几种元素（对应开始提到的三种机制）：

1. 基本数据和过程：数字和算术运算；
2. 通过嵌套组合生成组合操作；
3. 通过定义将变量名与值绑定，提供简单的抽象方法。

接下来要学习的是**过程定义（procedure definition）**，是一种更有力的抽象方法，可以对组合操作进行命名，并当做整体来操作：

In [12]:
(define (square x) (* x x))
(square 21)

441

In [13]:
(square (+ 2 5))

49

In [14]:
(square (square 3))

81

上例中我们得到一个**组合过程（compound procedure）**，并将其命名为`square`。过程定义的一般形式为：

```scm
(define (<name> <形参>) <过程体>)
```

组合过程与基本过程用起来是完全相同的，如此处我们定义的`square`与基本运算符`+`的使用方法是相同的。

#### 1.1.5 过程应用的替换模型

对组合过程的求值与1.1.3中提到到对组合式求值遵循相同的规则，将过程（运算符的值，可能是组合过程也可能是基本过程）应用到参数上。如果假设基本过程是内置在解释器里面与机器指令相关的，那么对于组合过程，其应用过程应该是：

> 将给定的参数绑定到形参，然后按照过程体的指令完成求值。

以下面的求值过程为例：

In [16]:
(define (sum-of-squares x y) (+ (square x) (square y)))
(define (f a)
  (sum-of-squares (+ a 1) (* a 2)))
(f 5)

136

我们可以通过**替换模型（substitution model）**来描述组合过程的求值流程：

```scm
(f 5)
(sum-of-squares (+ 5 1) (* 5 2))
(+ (square 6) (square 10))
(+ 36 100)
```

也就是用实际参数依次替换所有组合过程的过程体中的形参，最后化简为基本过程的组合，即可求出值。

---

对于这里所说的替换模型，需要注意以下两点：

1. 这种替换方法是用于帮助我们思考、理解组合过程的求值流程，而非解释器内部真实的运作原理；
2. 本书中将会涉及到许多不同的”模型“，用于阐述解释器的工作原理，但是开始接触到的往往是一些简化后的模型，我们将会在后面的章节中看到这些简化模型的不足之处，直到第五章最终实现一个完整的解释器。

---

**应用序和正则序**

按照1.1.3给出的求值规则，解释器首先对运算符和运算对象进行求值，然后将求值结果作为过程和参数，进一步求值。但这不是唯一的求值方法，我们将过程体、形参替换为实际参数，直到全部展开为只有基本过程组成的表达式，然后再进行计算求值，还是以上面的求值过程为例：

```scm
(f 5)
(sum-of-squares (+ 5 1) (* 5 2))
(+ (square (+ 5 1)) (square (* 5 2)))
(+ (* (+ 5 1) (+ 5 1))
   (* (* 5 2) (* 5 2)))
```

这种“*先完全展开然后逐步计算*”的演算方法称为**正则序求值（normal-order evaluation）**；解释器实际使用的“*先对参数求值，然后传递到过程*”演算方法称为**应用序求值（applicative-order evaluation）**。

#### 1.1.6 条件表达式和谓语

Lisp中条件表达式的一般形式如下：

```scm
(cond (<p1> <e1>)
     (<p2> <e2>)
     ...
     (<pn> <en>)
)
```

由`cond`语句和多对`(<p> <e>)`从句（clauses）组合而成，从句表达式的第一个元素（`<p>`）称为**谓语（predicate）**，谓语表达式的求值结果要么是`true`要么是`false`；从句的第二个元素称为**结果表达式（consequent expression）**。

条件表达式的求值规则如下：

1. 逐一对从句中的谓语求值，直到谓语返回结果为true；
2. 条件表达式的值就是谓语为true时对应的结果表达式的值；
3. 如果所有谓语求值均为false，条件表达式的结果为未定义。

例如计算绝对值：

In [2]:
(define (abs x)
  (cond ((> x 0) x)
        ((= x 0) 0)
        ((< x 0) (- x)))
  )
(abs -1)

1

另外一种计算方法如下所示：

In [3]:
(define (abs x)
  (cond ((< x 0) (- x))
        (else x)))
(abs -2)

2

这里用到特殊的`else`符号，可以放在条件表达式所有从句的最后，用来表示当前面所有谓语均为false时，返回最后一个结果表达式，实际上，任何结果永远为true的表达式均可以起到和`else`一样的作用。

还有另外一种计算绝对值的方法如下所示

In [4]:
(define (abs x)
  (if (< x 0)
      (- x)
      x))
(abs -3)

3

这里用到特殊形式`if`表达式，实际上相当于只有两个从句的条件表达式，其一般形式如下：

```scm
(if <谓语> <谓语为true返回的结果> <谓语为false返回的结果>)
```

除了基本谓语表达式如`<`，`=`和`>`等，还有一些其它的逻辑比较操作，可以用于组成复合式的谓语：

1. `(and <e1> ... <en>)`
2. `(or <e1> ... <en>)`
3. `(not <e>)`

`and`表达式的求值规则：

从左至右，依次对各个子表达式`<e>`求值，只要遇到`<e>`的值为fasle，则`and`表达式值为false，且**后面的子表达式不再继续求值**；只有当所有子表达式的值都是true，`and`表达式的值才为true。

`or`表达式的求值规则：

从左至右，依次对各个子表达式`<e>`求值，只要遇到`<e>`的值为true，则`and`表达式值为true，且**后面的子表达式不再继续求值**；只有当所有子表达式的值都是false，`and`表达式的值才为false。

`not`表达式的求值规则，**不是**真的就是假的。

---

需要注意的是，`and`和`or`属于特殊形式（**因为它们的子表达式并不一定会被求值**），而`not`才是一般的过程。

---

示例如下：

In [6]:
(define x 6)
(and (> x 5) (< x 10))

#t

In [7]:
(define (>= x y)
  (or (> x y) (= x y)))
(>= 6 6)

#t

In [8]:
(define (>= x y)
  (not (< x y)))
(>= 5 6)

#f